In [ ]:
import boto3

def describe_tags_for_resource(product_region_code, line_item_resource_id):
    """Fetch only the 'Key' and 'Value' attributes of tags for a given resource in a specific region."""
    try:
        # Initialize a Boto3 EC2 client for the specified region
        ec2_client = boto3.client('ec2', region_name=product_region_code)

        # Call the describe_tags method to get tags for the resource ID
        response = ec2_client.describe_tags(
            Filters=[
                {'Name': 'resource-id', 'Values': [line_item_resource_id]}
            ]
        )

        # Extract only the 'Key' and 'Value' attributes from tags
        tags = [{'Key': tag['Key'], 'Value': tag['Value']} for tag in response.get('Tags', [])]
        return tags

    except Exception as e:
        print(f"Error retrieving tags for resource {line_item_resource_id} in region {product_region_code}: {e}")
        return []

def describe_volumes_for_resource(product_region_code, line_item_resource_id):
    """Fetch volume information for a given resource id in a specific region."""
    try:
        ec2_client = boto3.client('ec2', region_name=product_region_code)
        response = ec2_client.describe_volumes(
            Filters=[
                {'Name': 'volume-id', 'Values': [line_item_resource_id]}
            ]
        )
        return response.get('Volumes', [])

    except Exception as e:
        print(f"Error retrieving volumes for resource {line_item_resource_id} in region {product_region_code}: {e}")
        return []

def describe_instances_for_resource(product_region_code, line_item_resource_id):
    """Fetch instance information for a given resource id in a specific region."""
    try:
        ec2_client = boto3.client('ec2', region_name=product_region_code)
        response = ec2_client.describe_instances(
            InstanceIds=[line_item_resource_id]
        )
        # Extracting instance information from response
        instances = response.get('Reservations', [])
        return instances

    except Exception as e:
        print(f"Error retrieving instances for resource {line_item_resource_id} in region {product_region_code}: {e}")
        return []

# Example list of triplets
triplets_example = [
    ["AmazonEC2", "us-west-1", "i-0a5d5c6cc752f8bea"],
    ["AmazonEBS", "us-east-1", "vol-0902bf12a36e57c9d"],
    ["AmazonEC2", "eu-west-1", "i-0b9473b3e68f5e9c5"],
    ["AmazonS3", "eu-central-1", "resource_id_4"]
]

# Initialize lists to collect results
aggregated_tags = []
aggregated_volumes = []
aggregated_instances = []

# Loop through triplets and use a switch-like structure to call appropriate functions
for triplet in triplets_example:
    product_code, region_code, resource_id = triplet

    if 'AmazonEC2' in product_code:
        tags = describe_tags_for_resource(region_code, resource_id)
        aggregated_tags.extend(tags)
        instances = describe_instances_for_resource(region_code, resource_id)
        aggregated_instances.extend(instances)
    elif 'AmazonEBS' in product_code:
        volumes = describe_volumes_for_resource(region_code, resource_id)
        aggregated_volumes.extend(volumes)
    else:
        print(f"No operation for product code: {product_code}")

# Output the aggregated results
print("Aggregated Tags:", aggregated_tags)
print("Aggregated Volumes:", aggregated_volumes)
print("Aggregated Instances:", aggregated_instances)

In [ ]:
import boto3
import re

def describe_tags_for_apigateway(region, resource_id):
    client = boto3.client('apigateway', region_name=region)
    try:
        response = client.get_tags(
            resourceArn=f"arn:aws:apigateway:{region}::/restapis/{resource_id}"
        )
        return response.get('tags', {})
    except Exception as e:
        print(f"Error retrieving tags for API Gateway {resource_id}: {e}")
        return {}

def describe_tags_for_cloudwatch(region, resource_arn):
    client = boto3.client('cloudwatch', region_name=region)
    try:
        response = client.list_tags_for_resource(ResourceARN=resource_arn)
        return response.get('Tags', [])
    except Exception as e:
        print(f"Error retrieving tags for CloudWatch {resource_arn}: {e}")
        return []

def describe_tags_for_docdb(region, resource_arn):
    client = boto3.client('docdb', region_name=region)
    try:
        response = client.list_tags_for_resource(ResourceName=resource_arn)
        return response.get('TagList', [])
    except Exception as e:
        print(f"Error retrieving tags for DocDB {resource_arn}: {e}")
        return []

def describe_tags_for_dynamodb(region, resource_arn):
    client = boto3.client('dynamodb', region_name=region)
    try:
        response = client.list_tags_of_resource(ResourceArn=resource_arn)
        return response.get('Tags', [])
    except Exception as e:
        print(f"Error retrieving tags for DynamoDB {resource_arn}: {e}")
        return []

def describe_tags_for_ec2_resource(region, resource_id):
    client = boto3.client('ec2', region_name=region)
    try:
        if re.match(r'^i-[a-f0-9]{17}$', resource_id):
            # Handle EC2 Instance
            response = client.describe_tags(
                Filters=[{'Name': 'resource-id', 'Values': [resource_id]}]
            )
        elif re.match(r'^vol-[a-f0-9]{17}$', resource_id):
            # Handle EBS Volume
            response = client.describe_tags(
                Filters=[{'Name': 'resource-id', 'Values': [resource_id]}]
            )
        elif re.match(r'^arn:aws:ec2:[a-z-]+-\d:[0-9]+:snapshot/snap-[a-f0-9]{17}$', resource_id):
            # Handle EBS Snapshot ARN
            response = client.describe_tags(
                Filters=[{'Name': 'resource-id', 'Values': [resource_id.split('/')[-1]]}]
            )
        elif re.match(r'^arn:aws:ec2:[a-z-]+-\d:[0-9]+:natgateway/nat-[a-f0-9]{17}$', resource_id):
            # Handle NAT Gateway ARN
            response = client.describe_tags(
                Filters=[{'Name': 'resource-id', 'Values': [resource_id.split('/')[-1]]}]
            )
        else:
            print(f"Unknown EC2 resource type for ID: {resource_id}")
            return []

        return [{'Key': tag['Key'], 'Value': tag['Value']} for tag in response.get('Tags', [])]

    except Exception as e:
        print(f"Error retrieving tags for EC2 resource {resource_id}: {e}")
        return []

def describe_tags_for_ecr(region, resource_arn):
    client = boto3.client('ecr', region_name=region)
    try:
        response = client.list_tags_for_resource(resourceArn=resource_arn)
        return response.get('tags', [])
    except Exception as e:
        print(f"Error retrieving tags for ECR {resource_arn}: {e}")
        return []

def describe_tags_for_ecs(region, resource_arn):
    client = boto3.client('ecs', region_name=region)
    try:
        response = client.list_tags_for_resource(resourceArn=resource_arn)
        return response.get('tags', [])
    except Exception as e:
        print(f"Error retrieving tags for ECS {resource_arn}: {e}")
        return []

def describe_tags_for_rds(region, resource_arn):
    client = boto3.client('rds', region_name=region)
    try:
        response = client.list_tags_for_resource(ResourceName=resource_arn)
        return response.get('TagList', [])
    except Exception as e:
        print(f"Error retrieving tags for RDS {resource_arn}: {e}")
        return []

def describe_tags_for_redshift(region, resource_arn):
    client = boto3.client('redshift', region_name=region)
    try:
        response = client.describe_tags(ResourceName=resource_arn)
        return response.get('TaggedResources', [])
    except Exception as e:
        print(f"Error retrieving tags for Redshift {resource_arn}: {e}")
        return []

def describe_tags_for_s3(region, bucket_name):
    client = boto3.client('s3', region_name=region)
    try:
        response = client.get_bucket_tagging(Bucket=bucket_name)
        return response.get('TagSet', [])
    except Exception as e:
        print(f"Error retrieving tags for S3 Bucket {bucket_name}: {e}")
        return []

def describe_tags_for_lambda(region, resource_arn):
    client = boto3.client('lambda', region_name=region)
    try:
        response = client.list_tags(Resource=resource_arn)
        return response.get('Tags', {})
    except Exception as e:
        print(f"Error retrieving tags for Lambda {resource_arn}: {e}")
        return {}

# Example triplets list
triplets_example = [
    ["AmazonApiGateway", "us-west-1", "resource_id_1"],
    ["AmazonCloudWatch", "us-east-1", "resource_arn_2"],
    ["AmazonDocDB", "eu-west-1", "resource_arn_3"],
    ["AmazonDynamoDB", "us-west-2", "resource_arn_4"],
    ["AmazonEC2", "us-west-1", "i-0a5d5c6cc752f8bea"],
    ["AmazonEC2", "us-east-1", "vol-0902bf12a36e57c9d"],
    ["AmazonEC2", "us-west-2", "arn:aws:ec2:us-west-2:123456789012:snapshot/snap-09e8ab2bdf5b3f0c5"],
    ["AmazonEC2", "us-east-2", "arn:aws:ec2:us-east-2:123456789012:natgateway/nat-0a12b345678cde890"],
    ["AmazonECR", "us-west-2", "resource_arn_9"],
    ["AmazonECS", "eu-central-1", "resource_arn_10"],
    ["AmazonRDS", "ap-southeast-1", "resource_arn_11"],
    ["AmazonRedshift", "us-east-1", "resource_arn_12"],
    ["AmazonS3", "us-west-2", "bucket_name_13"],
    ["AWSLambda", "us-east-1", "resource_arn_14"]
]

# Dictionary to map product codes to their respective functions
tag_functions = {
    "AmazonApiGateway": describe_tags_for_apigateway,
    "AmazonCloudWatch": describe_tags_for_cloudwatch,
    "AmazonDocDB": describe_tags_for_docdb,
    "AmazonDynamoDB": describe_tags_for_dynamodb,
    "AmazonEC2": describe_tags_for_ec2_resource,
    "AmazonECR": describe_tags_for_ecr,
    "AmazonECS": describe_tags_for_ecs,
    "AmazonRDS": describe_tags_for_rds,
    "AmazonRedshift": describe_tags_for_redshift,
    "AmazonS3": describe_tags_for_s3,
    "AWSLambda": describe_tags_for_lambda
}

# Loop through triplets and call the appropriate function
for triplet in triplets_example:
    product_code, region_code, resource_id = triplet

    if product_code in tag_functions:
        tags = tag_functions[product_code](region_code, resource_id)
        print(f"Tags for {resource_id} in {region_code}: {tags}")
    else:
        print(f"No tag function available for product code: {product_code}")